# LSTM Bot

## Project Overview

In this project, you will build a chatbot that can converse with you at the command line. The chatbot will use a Sequence to Sequence text generation architecture with an LSTM as it's memory unit. You will also learn to use pretrained word embeddings to improve the performance of the model. At the conclusion of the project, you will be able to show your chatbot to potential employers.

Additionally, you have the option to use pretrained word embeddings in your model. We have loaded Brown Embeddings from Gensim in the starter code below. You can compare the performance of your model with pre-trained embeddings against a model without the embeddings.



---



A sequence to sequence model (Seq2Seq) has two components:
- An Encoder consisting of an embedding layer and LSTM unit.
- A Decoder consisting of an embedding layer, LSTM unit, and linear output unit.

The Seq2Seq model works by accepting an input into the Encoder, passing the hidden state from the Encoder to the Decoder, which the Decoder uses to output a series of token predictions.

## Dependencies

- Pytorch
- Numpy
- Pandas
- NLTK
- Gzip
- Gensim


Please choose a dataset from the Torchtext website. We recommend looking at the Squad dataset first. Here is a link to the website where you can view your options:

- https://pytorch.org/text/stable/datasets.html





In [21]:
test = False
init = False

if test:
    !pip install -U  pytest
if init:
    pass
    #!pip install torchsummary
    #!pip install typing-extensions --upgrade
    #!pip install -U torch torchvision torchtext torchdata pytest

In [22]:
# public libraries
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import torch.nn as nn
import pandas as pd
import torch.optim
import random
from torch.utils.data import DataLoader

# my libraries
import src.utils
import src.data

In [23]:
if False:
    from nltk.corpus import brown
    from nltk.tokenize import RegexpTokenizer
    import gensim


    #from ntlk.stem import *

    nltk.download('brown')
    nltk.download('punkt')

    # Output, save, and load brown embeddings

    model = gensim.models.Word2Vec(brown.sents())
    model.save('brown.embedding')

    w2v = gensim.models.Word2Vec.load('brown.embedding')



    def loadDF(path):
      '''

      You will use this function to load the dataset into a Pandas Dataframe for processing.

      '''
      return df


    def prepare_text(sentence):

        '''

        Our text needs to be cleaned with a tokenizer. This function will perform that task.
        https://www.nltk.org/api/nltk.tokenize.html

        '''

        return tokens



    def train_test_split(SRC, TRG):

        '''
        Input: SRC, our list of questions from the dataset
                TRG, our list of responses from the dataset

        Output: Training and test datasets for SRC & TRG

        '''
        share = 0.2
        src_split = len(SRC)*share
        trg_split = len(TRG)*share
        SRC_test_dataset = SRC[:scr_split]
        SRC_train_dataset = SRC[scr_split:]
        TRG_test_dataset = SRC[:trg_split]
        TRG_train_dataset = SRC[trg_split:]


        return SRC_train_dataset, SRC_test_dataset, TRG_train_dataset, TRG_test_dataset


### Get and watch the data

In [24]:
if init:
    data.squad1_to_csv()
    
df_train = pd.read_csv('data/train_dataset_squad1.csv')  
df_test = pd.read_csv('data/dev_dataset_squad1.csv') 
split = round(len(df_train)*100/(len(df_train)+len(df_test)),1)
print(f"{len(df_train)} training samples and {len(df_test)} have been loaded.")
print(f"The test data makes {split}% of all the data.")
df_train.head()

87599 training samples and 10570 have been loaded.
The test data makes 89.2% of all the data.


,Unnamed: 0,context,question,answer,answer_start
0,0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,['Saint Bernadette Soubirous'],[515]
1,1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,['a copper statue of Christ'],[188]
2,2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,['the Main Building'],[279]
3,3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,['a Marian place of prayer and reflection'],[381]
4,4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,['a golden statue of the Virgin Mary'],[92]


### Create vocabulary and dictionary

In [25]:
vocab_source = src.utils.Vocab(name='source')
vocab_target = src.utils.Vocab(name='target')

questions = [vocab_source.clean_text(s[2:-2]) for s in df_train["answer"].values.tolist()]
answers = [vocab_target.clean_text(q[2:-2]) for q in df_train["question"].values.tolist()]


count = 0
for q, a in zip(questions, answers):

    PAD = "<PAD>"
    SOS = "<SOS>"
    EOS = "<EOS>"
    OUT = "<OUT>"
    special_tokens = ["<PAD>", "<SOS>", "<EOS>", "<OUT>"]
    for word in special_tokens + q:
        vocab_source.indexWord(word)
    for word in special_tokens + a:
        vocab_target.indexWord(word)

print("source:", len(vocab_source.words), "target:", len(vocab_target.words), "common words:", len(list(set(vocab_source.words.keys()).intersection(set(vocab_target.words.keys())))))

questions = src.utils.tokenize_questions(questions, vocab_source)
answers = src.utils.tokenize_answers(answers, vocab_target)


source: 42861 target: 49975 common words: 23297


In [26]:
questions.shape

torch.Size([87599, 45])

### Calculating sequence necessary sequence length

In [27]:
loader  = DataLoader(list(zip(questions, answers)), shuffle = False, batch_size = 30)
for a, b in loader:
    print(a.shape, b.shape)
    break

torch.Size([30, 45]) torch.Size([30, 41])


In [28]:

sequence_length_source = src.utils.get_sequence_length(questions)
sequence_length_target = src.utils.get_sequence_length(answers)
print(f"The source sequence has to have a length of {sequence_length_source}. And the target sequence has to have a length of {sequence_length_target}.")


The source sequence has to have a length of 45. And the target sequence has to have a length of 41.


### Tests

In [29]:
if test:
    !python -m pytest -vv tests.py

### Defining the model

In [30]:
import torch.nn as nn
class Encoder(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        
        super(Encoder, self).__init__()
        
        # self.embedding provides a vector representation of the inputs to our model
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        # self.lstm, accepts the vectorized input and passes a hidden state
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.dropout = nn.Dropout(p=0.0)
        
    
    def forward(self, i, h):
        
        '''
        Inputs: i, the src vector
        Outputs: o, the encoder outputs
                h, the hidden state (actually a tuple of hidden state and cell state)
        '''
        embedding = self.embedding(i)
        x,y = h
        o, h= self.lstm(embedding, h)
        o = self.dropout(o)
        
        return o, h
    

class Decoder(nn.Module):
      
    def __init__(self, hidden_size, output_size):
        
        super(Decoder, self).__init__()
        
        # self.embedding provides a vector representation of the target to our model
        self.embedding = nn.Embedding(output_size, hidden_size)
        
        # self.lstm, accepts the embeddings and outputs a hidden state
        self.lstm = nn.LSTM(hidden_size, hidden_size)

        # self.ouput, predicts on the hidden state via a linear output layer  
        self.linear = nn.Linear(hidden_size, output_size)
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, i, h):
        
        '''
        Inputs: i, the target vector
        Outputs: o, the decoder output
                h, the hidden state (actually a tuple of hidden state and cell state)
        '''

        embedding = self.embedding(i)

        o, h = self.lstm(embedding, h)

        o = self.linear(o)

        o = self.softmax(o)

        
        return o, h
        
        

class Seq2Seq(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size)
        self.decoder = Decoder(hidden_size, output_size)
                
        
    
    
    def forward(self, src, trg, start, teacher_forcing_ratio = 0.5): 
        '''
        Inputs: src, the source vector
                trg, the target vector
        Outputs: o, the prediction
                
        '''
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        src.to(device)
        trg.to(device)
        start.to(device)
        
            
        
        # encoder
        hidden = (torch.zeros(1,hidden_size).to(device), torch.zeros(1,hidden_size).to(device))
        for word in src:
            #print("word",word.shape,word)
            o, hidden = self.encoder(word.view(-1).to(device), hidden)
        
        # decoder
        o = start
        
        prediction = []
        for word in trg:
            #print("o", o)
            #print("o.view(-1)",o.view(-1))
            o, hidden = self.decoder(o.view(-1).to(device), hidden)
            prediction.append(o)

            if self.training:
                o = word if random.random() < teacher_forcing_ratio else torch.argmax(o,dim=1)
            else:
                o = torch.argmax(o,dim=1)
            

            
        
        return torch.stack(prediction).squeeze(1)

    



### Getting the data

### Training

In [31]:
questions.shape

torch.Size([87599, 45])

In [32]:
questions = src.utils.tokenize_questions(questions, vocab_source)
answers = src.utils.tokenize_answers(answers, vocab_target)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
is_cuda = torch.cuda.is_available()
#is_cuda = False
print("is cuda:",is_cuda)


# hyperparams
epochs = 1
batch_size = len(df_train)
batch_size = 32
hidden_size = 10
lr = 0.01
teacher_forcing_ratio = 0.5
loader  = DataLoader(list(zip(questions, answers)), shuffle = False, batch_size = batch_size)


# model
input_size = len(vocab_source.words)
hidden_size = hidden_size
output_size = len(vocab_target.words)
model = Seq2Seq(input_size, hidden_size, output_size)
model.to(device)
    
#
optim = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.NLLLoss()


# training

epoch = 0
for epoch in range(epochs):

    for batch_i, (questions, answers) in enumerate(loader):
        batch_loss = 0
        for q, a in zip(questions, answers):       
            start = next(iter(torch.LongTensor([vocab_target.words["<SOS>"]])))
            optim.zero_grad()
            output = model(q,a,start)
            
            rand = random.uniform(0,1)
            loss = loss_fn(output, a.to(device))
            loss.backward(retain_graph=True)
            optim.step()
            batch_loss += loss.item()
            #print("loss:",loss.item())
        if batch_i % 50 == 0:
            print(batch_i,batch_loss/batch_size)

            
        

is cuda: True
0 -1.956096156163767e-05
50 -1.952518056214103e-05
100 -1.9525384459484485e-05
150 -1.939486793389733e-05
200 -1.9587085773764557e-05
250 -1.9597147741023946e-05
300 -1.963699156704024e-05
350 -1.9657596169508906e-05
400 -1.9516624831794616e-05
450 -1.9582407844609406e-05
500 -1.9777185741531866e-05


KeyboardInterrupt: 

In [ ]:
print(qt)
print(trg)
print(hidden)

In [ ]:
torch.cuda.memory_summary(device='cuda', abbreviated=False)

In [ ]:
t = torch.Tensor(2)
#t = t.view(1,-1)
t.shape
t=t.view()
t


In [ ]:
at = at.view(-1)
output = output.squeeze(0)
print(at.shape)
print(output.shape)
loss_fn(output,at)

In [ ]:
# is_cuda = torch.cuda.is_available()
# print(is_cuda)

# import torch.optim
# # hyperparams
# epochs = 1
# batch_size = len(df_train)
# batch_size = 10
# hidden_size = 100
# lr = 0.01
# teacher_forcing_ratio = 0.5


# # model
# encoder_input_size = len(vocab.words)
# embedding_size = len(vocab.words)
# hidden_size = hidden_size
# decoder_output_size = len(vocab.words)
# print(encoder_input_size, embedding_size, hidden_size, decoder_output_size)


# model = Seq2Seq(encoder_input_size, embedding_size, hidden_size, decoder_output_size)
# if is_cuda:
#     model.cuda()
# optim = torch.optim.SGD(model.parameters(), lr=lr)
# loss_fn = nn.NLLLoss()


# # training
# batches = len(df_train)//batch_size
# epoch = 0
# for epoch in range(epochs):
#     hidden = (torch.zeros(1,1,hidden_size), torch.zeros(1,1,hidden_size))
#     for batch in range(batches):
                   
#         for q, a in zip(questions[:10], answers[:10]):
#             trg = torch.LongTensor([[vocab.words["<SOS>"]]])
#             print(trg, trg.shape)

#             for qt, at in zip(q, a):
                
#                 optim.zero_grad() 
#                 at = at.view(1,-1)
#                 qt = qt.view(1,-1)
#                 if is_cuda:
#                     at=at.cuda()
#                     qt=qt.cuda()
#                     trg=trg.cuda()
#                     a,b=hidden
#                     hidden = a.cuda(),b.cuda()
#                 print("trg", trg.shape)
#                 print("at", at.shape)
#                 output, hidden = model(qt, trg, hidden)


#                 a,b = hidden
#                 print(output.shape, at.shape)
#                 print(output, at)
                
#                 rand = random.uniform(0,1)
#                 if rand > teacher_forcing_ratio:
#                     loss = loss_fn(output, at)
#                 else:
#                     loss = loss_fn(output, trg)
#                 loss.backward()
#                 trg = output
#                 print("###")
            
            
            
#         break
#     break
        

In [ ]:
for que in questions[:2]:
    for word in que:
        print(word.shape, word.view(-1).shape,word.view(1,-1).shape,word.view(1,-1), word.view(1,1,-1).shape)

In [ ]:
import src.abc
src.abc.test()

In [ ]:
import torch.nn.functional as F
F.one_hot(torch.Tensor([19]),50)